In [ ]:
pip install roboflow supervision opencv-python

## Lewat Upload Gambar

In [2]:
from roboflow import Roboflow
import supervision as sv
import cv2
from tkinter import Tk, filedialog

# Fungsi untuk membuka dialog pemilihan file
def select_image_file():
    root = Tk()
    root.withdraw()  # Sembunyikan jendela utama tkinter
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    return file_path

# Pilih file gambar
image_path = select_image_file()
if not image_path:
    raise FileNotFoundError("Tidak ada file yang dipilih.")

# Inisialisasi Roboflow dengan API key
rf = Roboflow(api_key="NXXlYL4oWi6tiJjGY79O")
project = rf.workspace().project("botanify-hama")
model = project.version(5).model

# Prediksi menggunakan model
result = model.predict(image_path, confidence=40, overlap=30).json()

# Ekstraksi label dari hasil prediksi
labels = [item["class"] for item in result["predictions"]]

# Konversi hasil prediksi menjadi format Detections dari supervision
detections = sv.Detections.from_roboflow(result)

# Inisialisasi annotator
label_annotator = sv.LabelAnnotator()
box_annotator = sv.BoxAnnotator()  # Sesuaikan nama variabel

# Membaca gambar
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError("Gambar tidak ditemukan atau tidak bisa dimuat.")

# Anotasi gambar dengan bounding box
annotated_image = box_annotator.annotate(
    scene=image, detections=detections)
# Anotasi gambar dengan label
annotated_image = label_annotator.annotate(
    scene=annotated_image, detections=detections, labels=labels)

# Menampilkan gambar yang telah dianotasi
sv.plot_image(image=annotated_image, size=(16, 16))


## Pakai Kamera

In [ ]:
from roboflow import Roboflow
import supervision as sv
import cv2

# Fungsi untuk mengambil gambar dari kamera
def capture_image(filename="captured_image.jpg"):
    cap = cv2.VideoCapture(0)  # Menggunakan kamera default
    if not cap.isOpened():
        raise IOError("Tidak bisa membuka kamera")

    while True:
        ret, frame = cap.read()
        if not ret:
            raise IOError("Gagal menangkap gambar")

        cv2.imshow('Tekan Spasi untuk Memotret, Esc untuk Keluar', frame)
        
        key = cv2.waitKey(1)
        if key % 256 == 27:  # Tekan Esc untuk keluar
            print("Keluar tanpa memotret")
            break
        elif key % 256 == 32:  # Tekan Spasi untuk memotret
            cv2.imwrite(filename, frame)
            print(f"Gambar disimpan sebagai {filename}")
            break

    cap.release()
    cv2.destroyAllWindows()

# Memotret gambar
capture_image("captured_image.jpg")

# Inisialisasi Roboflow dengan API key
rf = Roboflow(api_key="M366UrrLGR5h4NZuVKCP")
project = rf.workspace().project("pest-detection-aphids-thrips-and-white-fly-in-rose-and-hibiscus-plants")
model = project.version(2).model

# Prediksi menggunakan model
result = model.predict("captured_image.jpg", confidence=40, overlap=30).json()

# Ekstraksi label dari hasil prediksi
labels = [item["class"] for item in result["predictions"]]

# Konversi hasil prediksi menjadi format Detections dari supervision
detections = sv.Detections.from_roboflow(result)

# Inisialisasi annotator
label_annotator = sv.LabelAnnotator()
box_annotator = sv.BoxAnnotator()  # Sesuaikan nama variabel

# Membaca gambar yang telah diambil
image = cv2.imread("captured_image.jpg")
if image is None:
    raise FileNotFoundError("Gambar tidak ditemukan atau tidak bisa dimuat.")

# Anotasi gambar dengan bounding box
annotated_image = box_annotator.annotate(
    scene=image, detections=detections)
# Anotasi gambar dengan label
annotated_image = label_annotator.annotate(
    scene=annotated_image, detections=detections, labels=labels)

# Menampilkan gambar yang telah dianotasi
sv.plot_image(image=annotated_image, size=(16, 16))


In [2]:
import requests
import supervision as sv
import cv2
from tkinter import Tk, filedialog
import base64
import json

# Fungsi untuk membuka dialog pemilihan file
def select_image_file():
    root = Tk()
    root.withdraw()  # Sembunyikan jendela utama tkinter
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    return file_path

# Pilih file gambar
image_path = select_image_file()
if not image_path:
    raise FileNotFoundError("Tidak ada file yang dipilih.")

# Membaca gambar sebagai binary dan encode ke base64
with open(image_path, "rb") as image_file:
    image_bytes = image_file.read()
    image_base64 = base64.b64encode(image_bytes).decode('utf-8')

# Inisialisasi IBM Watson dengan API key
API_KEY = "R1yulikIDJRlU7wzw8KoMAGvw06AGaTM0goLuJmXih0m"  # Ganti dengan API key Anda
token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey": API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]

# Header untuk request ke IBM Watson
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

# Payload untuk scoring
payload_scoring = {
    "input_data": [
        {
            "fields": ["image"],
            "values": [[image_base64]]  # Mengirim gambar dalam bentuk base64
        }
    ]
}

# URL dari deployment model di IBM Watson Studio
model_url = 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/2c6f0b55-c9ff-46c5-a3a3-c7ad5167cbda/predictions?version=2021-05-01'

# Melakukan request ke model untuk prediksi
response = requests.post(model_url, json=payload_scoring, headers=headers)
if response.status_code != 200:
    raise Exception(f"Error in prediction: {response.text}")

result = response.json()

# Ekstraksi label dan prediksi dari hasil
predictions = result["predictions"][0]["values"]
labels = [item["class"] for item in predictions]

# Konversi hasil prediksi menjadi format Detections dari supervision (asumsi format prediksi sesuai dengan yang diharapkan supervision)
detections = sv.Detections.from_roboflow(predictions)

# Inisialisasi annotator
label_annotator = sv.LabelAnnotator()
box_annotator = sv.BoxAnnotator()  # Sesuaikan nama variabel

# Membaca gambar
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError("Gambar tidak ditemukan atau tidak bisa dimuat.")

# Anotasi gambar dengan bounding box
annotated_image = box_annotator.annotate(scene=image, detections=detections)
# Anotasi gambar dengan label
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)

# Menampilkan gambar yang telah dianotasi
sv.plot_image(image=annotated_image, size=(16, 16))